In [22]:
import numpy as np
import pandas as pd
import string, xlrd, nltk, unicodedata, pickle, os, re, time
import xml.etree.cElementTree as ET
from nltk import word_tokenize
from collections import defaultdict, Counter
import difflib

from bs4 import BeautifulSoup

In [2]:
df = pd.DataFrame(np.random.randint(5, 10, size=10), columns=["Numbers"])
df

,Numbers
0,5
1,9
2,7
3,9
4,5
5,8
6,6
7,5
8,6
9,7


In [3]:
unique_rows = df.index.tolist()
i = 0
while i<len(unique_rows)-1:
    j = i+1
    while j<len(unique_rows):
        print ("i is", i, "and j is", j)
        print (df.Numbers[unique_rows[i]], df.Numbers[unique_rows[j]])
        if df.Numbers[i]==df.Numbers[j]:
            unique_rows = unique_rows[:j] + unique_rows[j+1:]
            print (unique_rows)
        else:
            j += 1
    i += 1
unique_rows

i is 0 and j is 1
5 9
i is 0 and j is 2
5 7
i is 0 and j is 3
5 9
i is 0 and j is 4
5 5
[0, 1, 2, 3, 5, 6, 7, 8, 9]
i is 0 and j is 4
5 8
[0, 1, 2, 3, 6, 7, 8, 9]
i is 0 and j is 4
5 6
[0, 1, 2, 3, 7, 8, 9]
i is 0 and j is 4
5 5
[0, 1, 2, 3, 8, 9]
i is 0 and j is 4
5 6
[0, 1, 2, 3, 9]
i is 0 and j is 4
5 7
[0, 1, 2, 3]
i is 1 and j is 2
9 7
i is 1 and j is 3
9 9
[0, 1, 2]


[0, 1, 2]

In [4]:
df.reindex(unique_rows)

,Numbers
0,5
1,9
2,7


In [5]:
df.reindex(unique_rows)

,Numbers
0,5
1,9
2,7


In [6]:
df2 = df.reindex([4, 5, 8])
df2

,Numbers
4,5
5,8
8,6


In [7]:
df

,Numbers
0,5
1,9
2,7
3,9
4,5
5,8
6,6
7,5
8,6
9,7


In [8]:
l2 = [1, 2, 3, 4, 5]
l2[:2] + l2[3:]

[1, 2, 4, 5]

In [9]:
df3 = pd.DataFrame(np.random.randint(10, 15, size=10))
df3

,0
0,10
1,13
2,10
3,10
4,11
5,14
6,14
7,11
8,13
9,13


In [10]:
df3.drop(df3.index[[4, 6]])
df3

,0
0,10
1,13
2,10
3,10
4,11
5,14
6,14
7,11
8,13
9,13


In [11]:
data = {'name': ['Jason', 'Molly', 'Tina', 'Jake', 'Amy'], 
        'year': [2012, 2012, 2013, 2014, 2014], 
        'reports': [4, 24, 31, 2, 3]}
data_f = pd.DataFrame(data, index = ['Cochice', 'Pima', 'Santa Cruz', 'Maricopa', 'Yuma'])
data_f

,name,reports,year
Cochice,Jason,4,2012
Pima,Molly,24,2012
Santa Cruz,Tina,31,2013
Maricopa,Jake,2,2014
Yuma,Amy,3,2014


In [12]:
data_f.drop(data_f.index[1])

,name,reports,year
Cochice,Jason,4,2012
Santa Cruz,Tina,31,2013
Maricopa,Jake,2,2014
Yuma,Amy,3,2014


In [13]:
a = pd.DataFrame(np.random.randint(50, 100, size=40).reshape(8, 5), index=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
                columns=['X1', 'X2', 'X3', 'X4', 'X5'])
a

,X1,X2,X3,X4,X5
A,50,58,94,91,98
B,78,70,92,90,96
C,72,89,66,70,75
D,59,61,72,77,50
E,96,73,93,68,57
F,88,72,61,74,88
G,72,79,98,85,90
H,50,77,85,80,93


In [14]:
a.drop(a.columns[3], axis=1).columns = [1, 2, 3, 4]


In [15]:
a

,X1,X2,X3,X4,X5
A,50,58,94,91,98
B,78,70,92,90,96
C,72,89,66,70,75
D,59,61,72,77,50
E,96,73,93,68,57
F,88,72,61,74,88
G,72,79,98,85,90
H,50,77,85,80,93


In [16]:
b = pd.DataFrame.copy(a)
b

,X1,X2,X3,X4,X5
A,50,58,94,91,98
B,78,70,92,90,96
C,72,89,66,70,75
D,59,61,72,77,50
E,96,73,93,68,57
F,88,72,61,74,88
G,72,79,98,85,90
H,50,77,85,80,93


In [17]:
a = a.drop('X3', axis=1)
a

,X1,X2,X4,X5
A,50,58,91,98
B,78,70,90,96
C,72,89,70,75
D,59,61,77,50
E,96,73,68,57
F,88,72,74,88
G,72,79,85,90
H,50,77,80,93


In [18]:
print(b)
b.ix[4].Name

   X1  X2  X3  X4  X5
A  50  58  94  91  98
B  78  70  92  90  96
C  72  89  66  70  75
D  59  61  72  77  50
E  96  73  93  68  57
F  88  72  61  74  88
G  72  79  98  85  90
H  50  77  85  80  93


AttributeError: 'Series' object has no attribute 'Name'

In [21]:
b.drop([b.index[4]])

,X1,X2,X3,X4,X5
A,50,58,94,91,98
B,78,70,92,90,96
C,72,89,66,70,75
D,59,61,72,77,50
F,88,72,61,74,88
G,72,79,98,85,90
H,50,77,85,80,93


In [1]:
set([1,2,3,4,5,5,5,5,6,7,8])

{1, 2, 3, 4, 5, 6, 7, 8}